In [1]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [3]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [5]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Kendrick Lamar']

In [6]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1483 entries, 1283 to 5403
Data columns (total 5 columns):
song_owner      1483 non-null object
song_title      1483 non-null object
song_element    1483 non-null object
verse           1483 non-null object
artist          1483 non-null object
dtypes: object(5)
memory usage: 69.5+ KB


In [7]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [8]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
1283,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],"Lookin' in the mirror, I'm embarrassed, I'm fe...",Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i..."
1806,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],"Lookin' in the mirror, I'm embarrassed, I'm fe...",Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i..."
2890,Kendrick Lamar,Look Out For Detox,[Intro],"Uh, shoutout to ScHoolboy Q, He uglier than a ...",Kendrick Lamar,"[uh, shoutout, to, schoolboy, q, he, uglier, t..."
2891,Kendrick Lamar,Look Out For Detox,[Verse],"Tire marks, tire marks, Finish line with the t...",Kendrick Lamar,"[tire, marks, tire, marks, finish, line, with,..."
2892,Kendrick Lamar,HUMBLE.,[Intro],"Nobody pray for me, It's been that day for me,...",Kendrick Lamar,"[nobody, pray, for, me, it's, been, that, day,..."


In [9]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [10]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1283,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],"Lookin' in the mirror, I'm embarrassed, I'm fe...",Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i...","[lookin, ', in, the, mirror, ,, i, 'm, embarra..."
1806,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],"Lookin' in the mirror, I'm embarrassed, I'm fe...",Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i...","[lookin, ', in, the, mirror, ,, i, 'm, embarra..."
2890,Kendrick Lamar,Look Out For Detox,[Intro],"Uh, shoutout to ScHoolboy Q, He uglier than a ...",Kendrick Lamar,"[uh, shoutout, to, schoolboy, q, he, uglier, t...","[uh, ,, shoutout, to, schoolboy, q, ,, he, ugl..."
2891,Kendrick Lamar,Look Out For Detox,[Verse],"Tire marks, tire marks, Finish line with the t...",Kendrick Lamar,"[tire, marks, tire, marks, finish, line, with,...","[tire, marks, ,, tire, marks, ,, finish, line,..."
2892,Kendrick Lamar,HUMBLE.,[Intro],"Nobody pray for me, It's been that day for me,...",Kendrick Lamar,"[nobody, pray, for, me, it's, been, that, day,...","[nobody, pray, for, me, ,, it, 's, been, that,..."


In [11]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [12]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

1283    lookin in the mirror im embarrassed im feelin ...
1806    lookin in the mirror im embarrassed im feelin ...
2890    shoutout to schoolboy q he uglier than a mothe...
2891    tire marks tire marks finish line with the tir...
2892    nobody pray for me its been that day for me wa...
Name: verse, dtype: object

In [13]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1283,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],lookin in the mirror im embarrassed im feelin ...,Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i...","[lookin, ', in, the, mirror, ,, i, 'm, embarra..."
1806,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],lookin in the mirror im embarrassed im feelin ...,Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i...","[lookin, ', in, the, mirror, ,, i, 'm, embarra..."
2890,Kendrick Lamar,Look Out For Detox,[Intro],shoutout to schoolboy q he uglier than a mothe...,Kendrick Lamar,"[uh, shoutout, to, schoolboy, q, he, uglier, t...","[uh, ,, shoutout, to, schoolboy, q, ,, he, ugl..."
2891,Kendrick Lamar,Look Out For Detox,[Verse],tire marks tire marks finish line with the tir...,Kendrick Lamar,"[tire, marks, tire, marks, finish, line, with,...","[tire, marks, ,, tire, marks, ,, finish, line,..."
2892,Kendrick Lamar,HUMBLE.,[Intro],nobody pray for me its been that day for me wa...,Kendrick Lamar,"[nobody, pray, for, me, it's, been, that, day,...","[nobody, pray, for, me, ,, it, 's, been, that,..."


In [14]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [15]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1283,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],lookin in the mirror im embarrassed im feelin ...,Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i...","[lookin, ', in, the, mirror, ,, i, 'm, embarra..."
1806,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],lookin in the mirror im embarrassed im feelin ...,Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i...","[lookin, ', in, the, mirror, ,, i, 'm, embarra..."
2890,Kendrick Lamar,Look Out For Detox,[Intro],shoutout to schoolboy q he uglier than a mothe...,Kendrick Lamar,"[uh, shoutout, to, schoolboy, q, he, uglier, t...","[uh, ,, shoutout, to, schoolboy, q, ,, he, ugl..."
2891,Kendrick Lamar,Look Out For Detox,[Verse],tire marks tire marks finish line with the tir...,Kendrick Lamar,"[tire, marks, tire, marks, finish, line, with,...","[tire, marks, ,, tire, marks, ,, finish, line,..."
2892,Kendrick Lamar,HUMBLE.,[Intro],nobody pray for me its been that day for me wa...,Kendrick Lamar,"[nobody, pray, for, me, it's, been, that, day,...","[nobody, pray, for, me, ,, it, 's, been, that,..."


In [16]:
get_words(model_df)

['lookin',
 'in',
 'the',
 'mirror',
 'im',
 'embarrassed',
 'im',
 'feelin',
 'like',
 'a',
 'suicidal',
 'terrorist',
 'react',
 'like',
 'an',
 'infant',
 'whenever',
 'you',
 'are',
 'mentioned',
 'mind',
 'over',
 'matter',
 'never',
 'worked',
 'for',
 'my',
 'nemesis',
 'im',
 'in',
 'the',
 'matter',
 'of',
 'man',
 'arm',
 'wrestlin',
 'hands',
 'i',
 'was',
 'dealt',
 'when',
 'i',
 'said',
 'the',
 'music',
 'business',
 'was',
 'all',
 'i',
 'needed',
 'when',
 'i',
 'got',
 'it',
 'i',
 'was',
 'greeted',
 'by',
 'an',
 'alien',
 'that',
 'said',
 'last',
 'year',
 'that',
 'she',
 'slept',
 'with',
 'a',
 'canadian',
 'that',
 'gave',
 'him',
 'an',
 'addiction',
 'thatd',
 'keep',
 'him',
 'in',
 'mercedes',
 'benz',
 'bright',
 'lights',
 'and',
 'rihanna',
 'as',
 'a',
 'lady',
 'friend',
 'my',
 'vice',
 'is',
 'similar',
 'women',
 'love',
 'when',
 'youre',
 'my',
 'type',
 'and',
 'youre',
 'winnin',
 'from',
 'everything',
 'that',
 'your',
 'palm',
 'write',
 'pu

In [17]:
verses = list(model_df.verse.values)

In [18]:
verses[:10]

['lookin in the mirror im embarrassed im feelin like a suicidal terrorist react like an infant whenever you are mentioned mind over matter never worked for my nemesis im in the matter of man arm wrestlin hands i was dealt when i said the music business was all i needed when i got it i was greeted by an alien that said last year that she slept with a canadian that gave him an addiction thatd keep him in mercedes benz bright lights and rihanna as a lady friend my vice is similar women love when youre my type and youre winnin from everything that your palm write put her in the palms hotel sin city devil in a dress platinum chanel live the ambiance all cause the audience one day said i would do it so instead of a verse bein read ima go and get some head off the strength of my music i tell a bad bitch yo ass too fat capitalize that and your weave look good with the indian tracks trackin device on your used series i dont call back just blame it on your canadian the same day we say were in th

In [19]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [20]:
count, following = make_word_dictionary(verses)

In [21]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [22]:
verses_bigrams = create_bigrams(verses)

In [23]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [24]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seeds)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

blue but if i kill them with collision kings did it i got the burner stashed to impress me askin where its at only upsets me you cant down me but an enemigo


people know just what you doin just make it to your ass on the streets only should know about them hockey pucks skating on why you emotional ah bitch you be hollering that


brother cry i got a swimming pool left stroke right stroke whats the yams whats the yams whats the best rapper alive jigga man should ve retired i m in the pudding and


bitch i woke up in weeks and i run at ho with a line for the instrumentals listen to you nigga got some more words i know you love french tip and trip


apple jacks and after that i m in it record twenty four inch rims put that pussy ah i dont save even with ten weapons of your touch adrenaline pump give me what


friends so he did a lot bout each other wrong thats why im by your rule nobody holding your gun was born with the bank today and told them color me bad blue


blue faces get that ne

liquor and put that on somethin 26 inch rims put that on my caller id pawning my chain in the disc player tell ho ride to it die to it ride to it


bitch a square trying to keep it k dot off at his best friend the fuck cares even if muslims is seen in my ear play with me nobody prayin for me i


liquor and tell my salary come a lump sum of hundreds dont talk about it i love you i could never really shoot back could of lost our life over hood rats we


blue faces i hit the prez with a white collar so why fast forward hot dollah had the vodka on the top of iv s on top say im me im just sayin


streets i forgot dont kill my vibe bitch dont kill my vibe bitch dont kill my vibe bitch dont kill my vibe bitch dont kill my vibe bitch dont kill my vibe bitch


friends bitch i need evidently all i all i got options like an erection with no condom this aint eazy i cant fuck with but they hate to work on this day forth


drugs wont get you killed my homeboy and god wall telling you that a way with

brother was talking bout nothing yakity yakity pushing my buttons like plaid shirts clapping them rapidly actually kurt co bain ten years in my way let the clipse distribute the work should rise

In [25]:
columns = ['markov_generated_verses']

kendrick_markov_df = pd.DataFrame(generated_raps, columns=columns)

In [26]:
kendrick_markov_df['artist'] = 'Kendrick Lamar'

kendrick_markov_df

,markov_generated_verses,artist
0,blue but if i kill them with collision kings d...,Kendrick Lamar
1,people know just what you doin just make it to...,Kendrick Lamar
2,brother cry i got a swimming pool left stroke ...,Kendrick Lamar
3,bitch i woke up in weeks and i run at ho with ...,Kendrick Lamar
4,apple jacks and after that i m in it record tw...,Kendrick Lamar
5,friends so he did a lot bout each other wrong ...,Kendrick Lamar
6,blue faces get that new money and power respec...,Kendrick Lamar
7,love speed up breed up or speed up dont you as...,Kendrick Lamar
8,blue pill then feel free my mind or die from n...,Kendrick Lamar
9,blue pill and remy red come through flow sicke...,Kendrick Lamar


In [27]:
with open('data/kendrick_markov_raps.pkl', 'wb') as picklefile:
    pickle.dump(kendrick_markov_df, picklefile)